# Project Overview

Health care economists question whether the benefits of added medical care outweigh the costs. [Almond et al. (2008](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2903901/) propose a Regression Discontinuity Design (RDD) to compare health outcomes of newborns born around the threshold of 1500 grams. Those born under 1500 grams are considered very low birth weight and are consequently receive supplemental medical attention. Those born above the threshold are generally considered healthy. Notably, 1500 grams is a conventional measure not necessarily rooted in biology. 

This project provides an overview of RDD, evaluates its assumptions, re-produces findings from Almond et al., and presents thoughts on increased medical expenditures.  

# Regression Discontinuity Design Primer

## Premise

RDD is common in political science and econometrics as a quasi-experimental design when randomization is not possible. The idea is to exploit a threshold in which units above and below the margin are assigned to "treatment" and "control" groups. Units that lie very close to the threshold are similar enough, on average, which allows researchers to identify the missing counterfactual required to estimate average treatment effects. As a result they may not vary much in terms of potential outcomes, and any differences in outcomes may be attributed to an intervention.

## Assumption

The key assumption for RDD to be valid is that the mean potential outcomes are continuous on the running variable. In other words, there must not be any discontinuous jump in our outcome variable when it is plotted against the variable that determines assignment to treatment or control plotted on the x axis. 

# Loading and Checking Data